- nagoyaScrape(startPage, lastPage) will scrape the site and save CSV on Google Drive
- nagoya_authenticate(project_id) will setup project and SQL on GCP
- connect() will create database and insert data
- cleanrp() will delete database and move the file to bk folder
- slee(hour) is sleep command


In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

def nagoyaScrape(startPage, lastPage):
    from bs4 import BeautifulSoup
    import requests, random, time, csv, os.path, datetime, sys, sqlalchemy
    # define output file name
    date = datetime.datetime.now().strftime("%Y%m%d_%H%M")
    global filename
    filename = f'/content/drive/My Drive/Nagoya_{date}.csv'
    for j in range(startPage, lastPage +1):
        print('Started page ' + str(j))
        url = 'https://sub.nagoya-shouhinken.com/prd/c_app/shoplists?page=' + str(j)
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser') #page変数はResponse．BeautifulSoup()にHTMLの内容が文字列として取り出して渡す

        names = soup.find_all("h2", class_="p-area-shoplist__headline")
        areas = soup.find_all("dd", class_="-area")
        categories = soup.find_all("dd", class_="-category")
        addresses = soup.find_all("dl", "p-area-shoplist__address")


        l = []
        d = {}
        for i in range(len(names)):
            name = names[i].text.split('\n')[1].strip()
            area = areas[i].text
            category = categories[i].text
            address = addresses[i].text.split("\n")[3]
            tel = addresses[i].text.split("\n")[-3]
            d = {"name": name, "area": area, "category": category, "address": address, "tel": tel}
            l.append(d)
    
        keys = l[0].keys()
        with open(filename, 'a', newline='') as nagoya:
            dict_writer = csv.DictWriter(nagoya, keys)
            dict_writer.writerows(l)
        print('Completed page '+ str(j))     
        time.sleep(random.randint(3,5)) #次のスクレイぷに進むまで5〜8秒ランダムに待つ
    print("Completed all scrape")
    return

Mounted at /content/drive


In [2]:
startPage=1
lastPage=2
nagoyaScrape(startPage, lastPage)

Started page 1
Completed page 1
Started page 2
Completed page 2
Completed all scrape


In [3]:
def nagoya_authenticate(project_id): #Your GCP project ID 
    # Authenticate user
    from google.colab import auth
    auth.authenticate_user()

    # Configure gcloud.
    !gcloud config set project {project_id}
    return
nagoya_authenticate("cospa101")

Updated property [core/project].


In [4]:
def connect():
    # grant Cloud SQL Client role to authenticated user
    current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"

    project_id = "cospa101"
    !gcloud projects add-iam-policy-binding {project_id} \
        --member=user:{current_user[0]} \
        --role="roles/cloudsql.client"

    # enable Cloud SQL Admin API
    !gcloud services enable sqladmin.googleapis.com  

    # PGoogle Cloud region and name for your Cloud SQL instance
    region = "us-central1" 
    instance_name = "cospa101" 

    # check if Cloud SQL instance exists in the provided region
    database_version = !gcloud sql instances describe {instance_name} --format="value(databaseVersion)"
    if database_version[0].startswith("MYSQL"):
        print("Found existing MySQL Cloud SQL Instance!")
    else:
        print("Creating new Cloud SQL instance...")
        password = input("Please provide a password to be used for database 'root' user: ")
        !gcloud sql instances create {instance_name} --database-version=MYSQL_8_0 \
            --region={region} --cpu=1 --memory=4GB --root-password={password} \
            --database-flags=cloudsql_iam_authentication=On

    # Create database
    !gcloud sql databases create nagoya --instance={instance_name}

    # Create a database user
    !gcloud sql users create user \
        --instance={instance_name} \
        --password="user"


    # install dependencies
    import sys
    !{sys.executable} -m pip install cloud-sql-python-connector["pymysql"] SQLAlchemy==2.0.7

    # initialize parameters
    INSTANCE_CONNECTION_NAME = f"{project_id}:{region}:{instance_name}" # i.e demo-project:us-central1:demo-instance
    print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
    DB_USER = "user"
    DB_PASS = "user"
    DB_NAME = "nagoya"

    from google.cloud.sql.connector import Connector
    import sqlalchemy

    # initialize Connector object
    connector = Connector()


    # function to return the database connection object
    def getconn():
        conn = connector.connect(
            INSTANCE_CONNECTION_NAME,
            "pymysql",
            user=DB_USER,
            password=DB_PASS,
            db=DB_NAME
        )
        return conn

    # create connection pool with 'creator' argument to our connection object function
    pool = sqlalchemy.create_engine(
        "mysql+pymysql://",
        creator=getconn,
    )



    # connect to connection pool
    with pool.connect() as db_conn:
        # create nagoya table in our cospa101 database
        db_conn.execute(
            sqlalchemy.text(
                "CREATE TABLE IF NOT EXISTS nagoya "
                "(name VARCHAR(255) NOT NULL, "
                "area VARCHAR(255) NOT NULL, category VARCHAR(255) NOT NULL, "
                "address VARCHAR(255) NOT NULL, tel VARCHAR(255) NOT NULL );"
            )
        )

        # commit transaction (SQLAlchemy v2.X.X is commit as you go)
        db_conn.commit()
        # insert data into our nagoya table
        insert_stmt = sqlalchemy.text(
                "INSERT INTO nagoya (name, area, category, address, tel) VALUES (:name, :area, :category, :address, :tel)",
        )


        import csv
        with open(filename, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                db_conn.execute(insert_stmt, parameters={"name":row[0], "area":row[1], "category":row[2], "address":row[3], "tel":row[4]})

            # commit transactions
            db_conn.commit()

    # cleanup connector object
    connector.close()

    # try code
    code = "select count(*) from nagoya;"
    with pool.connect() as db_conn:
        print(db_conn.execute(sqlalchemy.text(code)).fetchall())
    
    return

connect()

Updated IAM policy for project [cospa101].
bindings:
- members:
  - serviceAccount:service-506786559931@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:service-506786559931@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:506786559931@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-506786559931@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:service-506786559931@gcf-admin-robot.iam.gserviceaccount.com
  role: roles/cloudfunctions.serviceAgent
- members:
  - serviceAccount:service-506786559931@gcp-sa-cloudscheduler.iam.gserviceaccount.com
  role: roles/cloudscheduler.serviceAgent
- members:
  - user:yuki.ama8@gmail.com
  role: roles/cloudsql.client
- members:
  - user:yuki.ama8@gmail.com
  role: roles/cloudsql.instanceUser
- members

In [8]:
def cleanup():
    # If there is no existing database, delete none.
    !gcloud sql databases delete nagoya --instance=cospa101 --quiet
    """
    # delete Cloud SQL instance
    instance_name = "cospa101" 
    !gcloud sql instances delete {instance_name}
    """
    # Move the file to a backup directory
    from os import system
    system(f'mv "{filename}" "/content/drive/My Drive/bk_cospa101"')

    return
cleanup()

Deleted database [nagoya].


In [9]:
def sleep(hour):
    import time
    time.sleep(int(hour) * 360)
sleep(23)